# Table of Contents
1. [Travelling Salesman problem](#Lab-6-Optimization-Exercise:-Travelling-Salesman-problem)
2. [Multi-objective optimization](#Lab-7-Optimization-exercises:-Multi-objective-optimization)

# Lab 6 Optimization Exercise: Travelling Salesman problem

Implement the optimization algorithms on the Travelling salesman problem:
- greedy algorithm
- simulated annealing
- variable neighbourhood search
- local search

## Travelling Salesman Problem

Cities are consecutively numbered: $1,2,...,n$

We encode the solutions as $x=(x_1,x_2,...,x_n,x_1)$ where:

* $x_1$ is the index of the home city
* $x_i$ is the index of the $i^{th}$ city visited along the way
* $x_n$ is the last city visited
* Every city must be visited exactly once

In [1]:
import numpy as np
import timeit
import plotly.graph_objects as go

from numpy import sqrt
from numpy.random import Generator, PCG64

In [2]:
def simulateCities(rg, n_cities):
    cities = {}
    for j in range(n_cities):
        cities[j] = tuple(rg.random(size=2).round(2)) # we use round(2) to make the digits more human readable
    return cities

def drawSalesman(path, cities, title="Path taken"):
    # loop through the coordinates using the path sequence
    x = [cities[p][0] for p in path]
    y = [cities[p][1] for p in path]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, name='path', mode='lines'))
    fig.add_trace(go.Scatter(x=x, y=y, name="cities", mode='markers+text', 
        marker={'size': 10}, text=[str(i) for i in path], textposition="bottom center",))
    fig.update_layout(template="presentation", title=title, xaxis_title="X", yaxis_title="Y", width=800, height=600)
    fig.update_xaxes(range=[0, 1])
    fig.update_yaxes(range=[0, 1])
    fig.show() # to reveal the figure
    
def evaluate(path, cities):
    # measure the total distance travelled
    distance = 0
    for n, p in enumerate(path[:-1]): # we loop until the last city -1
        x1, y1 = cities[p]
        x2, y2 = cities[path[n+1]]
        length = sqrt((x2-x1)**2 + (y2-y1)**2)
        distance += length
    
    return distance

In [3]:
def generateNewCitySeq_fe(rg, path):
    new_path = np.array(path.copy()) # make a copy of path
    rg.shuffle(new_path[1:-1])
    return list(new_path)

def FullEnumeration(rg, cities, iterations):
    tracker = [] # a list of tuple to track the min distance
    path = list(range(len(cities))) + [0]
    distance_travelled = evaluate(path, cities) # calculate the inital objective function
    
    min_distance_travelled = distance_travelled # save the current solution as the "best solution"
    best_solution = path                        # save the current solution as the "best solution"
    
    print("running algorithm...")
    start_time = timeit.default_timer() # implement a timer to track running time
    for i in range(1, iterations):
        path = generateNewCitySeq_fe(rg, path)
        distance_travelled = evaluate(path, cities)
        if distance_travelled < min_distance_travelled:
            min_distance_travelled = distance_travelled # update the best solution if the distance is shorter
            best_solution = path
        tracker.append((i, min_distance_travelled, distance_travelled))
    print("algorithm end.")
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = go.Figure()
    x, y, y2 = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'))
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'))
    fig.update_layout(
        title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    fig.show()
    
    return best_solution, min_distance_travelled

In [12]:
# run
our_seed=1000
rg = Generator(PCG64(our_seed)) 
n_cities = 10
cities = simulateCities(rg, n_cities)
solution, distance = FullEnumeration(rg, cities, iterations=5000)
print("best solution:", solution)
print("distance travelled:", distance)

drawSalesman(solution, cities)

running algorithm...
algorithm end.
time taken :0.24434s


best solution: [0, 7, 5, 2, 1, 8, 4, 9, 6, 3, 0]
distance travelled: 2.8749313329677086


## Greedy Search

In [10]:
def get_distance(city_from, city_to, cities):
        x1, y1 = cities[city_from]
        x2, y2 = cities[city_to]
        return sqrt((x1-x2)**2 + (y1-y2)**2)
        
def GreedySearch(rg, cities):
    print("running algorithm...")
    start_time = timeit.default_timer() 
    gs_solution = [0]
    n_cities = len(cities)
    unassigned_cities = list(range(1, n_cities))
    while len(unassigned_cities)>0:
        last_city = gs_solution[-1]
        d_map = {i: get_distance(last_city, i, cities) for i in unassigned_cities}
        next_city = min(d_map.items(), key=lambda x: x[1])[0]
        gs_solution.append(next_city)
        unassigned_cities.remove(next_city)
    gs_solution.append(0)  # must return to home
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    return gs_solution, evaluate(gs_solution, cities)

##Local Search

In [6]:
def generateNewCitySeq_ls(path, c2, c1):
    new_path = path.copy() # make a copy of path H-1-2-3-...-H
    
    if c1 < c2:
        c1, c2 = c2, c1 # swap c1 and c2 if c1 < c2
    
    new_path[c1] = path[c2] # swap location of c1 and c2
    new_path[c2] = path[c1]
    
    # 2-OPT
    j = 0
    while j < abs(c1-c2)/2: # iterate across inbetween cities
        new_path[c2+j] = path[c1-j]
        new_path[c1-j] = path[c2+j]
        j = j + 1
    
    return new_path

def LocalSearch(rg, cities, iterations):
    tracker = [] # a list of tuple to track the min distance
    path = list(range(len(cities))) + [0]
    distance_travelled = evaluate(path, cities) # calculate the inital objective function
    
    min_distance_travelled = distance_travelled # save the current solution as the "best solution"
    best_solution = path                        # save the current solution as the "best solution"
    
    print("running algorithm...")
    start_time = timeit.default_timer() # implement a timer to track running time
    for i in range(1, iterations):
        
        c2 = rg.choice(range(1, len(cities)-1)) # pick a random city
        
        for c1 in range(c2+1, len(cities)): # step through each neighbor one at a time
            path = generateNewCitySeq_ls(path, c2, c1) # use 2-OPT method
            distance_travelled = evaluate(path, cities)
            tracker.append((i, min_distance_travelled, distance_travelled))
            
            if distance_travelled < min_distance_travelled:
                min_distance_travelled = distance_travelled # update the best solution if the distance is shorter
                best_solution = path
                break # end loop and go to next iteration if f(x^+)<f(x_c)
                 
                
    print("algorithm end.")
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = go.Figure()
    x, y, y2 = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'))
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'))
    fig.update_layout(title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    
    fig.show()
    
    return best_solution, min_distance_travelled

##Simulated Annealing

In [7]:
from plotly.subplots import make_subplots
def generateNewCitySeq_sa(path, c2, c1):
    new_path = path.copy() # make a copy of path H-1-2-3-...-H
    
    if c1 < c2:
        c1, c2 = c2, c1 # swap c1 and c2 if c1 < c2
    
    new_path[c1] = path[c2] # swap location of c1 and c2
    new_path[c2] = path[c1]
    
    # 2-OPT
    j = 0
    while j < abs(c1-c2)/2: # iterate across inbetween cities
        new_path[c2+j] = path[c1-j]
        new_path[c1-j] = path[c2+j]
        j = j + 1
    
    return new_path


def temperature_linear(init, i, iterations):
    # linear temperature decrease
    return init * (1-i/iterations)

def temperature_nonlinear(init, i, iterations):
    p_0 = 0.999
    p_f = 0.001
    return -init/np.log(p_0+(p_f-p_0)/iterations*i)
    

def SimulatedAnnealing(rg, cities, iterations, t_init=1):
    tracker = [] # a list of tuple to track the min distance
    path = list(range(len(cities))) + [0]
    distance_travelled = evaluate(path, cities) # calculate the inital objective function
    
    min_distance_travelled = distance_travelled # save the current solution as the "best solution"
    best_solution = path                        # save the current solution as the "best solution"
    
    print("running algorithm...")
    start_time = timeit.default_timer() # implement a timer to track running time
    state = path # to track the current state
    for i in range(1, iterations):
        
        c2 = rg.choice(range(1, len(cities)-1)) # pick a random city
        c1 = rg.choice(range(1, len(cities)-1)) # pick a random neighbour
        while c1 == c2:
            c1 = rg.choice(range(1, len(cities)-1)) # pick another if c1=c2
        
        path = generateNewCitySeq_sa(state, c2, c1) # use 2-OPT method
        distance_travelled = evaluate(path, cities)
        T = temperature_nonlinear(t_init, i, iterations)
        tracker.append((i, min_distance_travelled, distance_travelled, T))
        d = distance_travelled - min_distance_travelled
        if d < 0:
            min_distance_travelled = distance_travelled # update the best solution if the distance is shorter
            best_solution = path
            state = path
        else:
            r = rg.random()
            if r < np.exp(-d/T):
                state = path # only update state
                
                
    print("algorithm end.")
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    x, y, y2, y3 = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'), secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'), secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', name='Temperature'), secondary_y=True)
    fig.update_layout(
        title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    fig.show()
    
    return best_solution, min_distance_travelled

##VNS

In [8]:
from plotly.subplots import make_subplots

def VNS(rg, cities, iterations, K=5):
    tracker = [] # a list of tuple to track the min distance
    path = list(range(len(cities))) + [0]
    distance_travelled = evaluate(path, cities) # calculate the inital objective function
    
    min_distance_travelled = distance_travelled # save the current solution as the "best solution"
    best_solution = path                        # save the current solution as the "best solution"
    
    print("running algorithm...")
    start_time = timeit.default_timer() # implement a timer to track running time
    k=0 # initialize 
    for i in range(1, iterations):
        k = k + 1
        c2 = rg.choice(range(1, len(cities)-1)) # pick a random city
        
        # define the neighbourhood structure
        N = list(range(c2, c2+1+k))
        neighbours = [n for n in N if (n>0 and n<len(cities) and n!=c2)]
        
        for c1 in neighbours: # step through each neighbor one at a time
            path = generateNewCitySeq_ls(path, c2, c1) # use 2-OPT method
            distance_travelled = evaluate(path, cities)
            tracker.append((i, min_distance_travelled, distance_travelled, k))
            
            if distance_travelled < min_distance_travelled:
                min_distance_travelled = distance_travelled # update the best solution if the distance is shorter
                best_solution = path
                k=0
                break # end loop and go to next iteration if f(x^+)<f(x_c)
        if k == K:
            print('k reached')
            break
                
    print("algorithm end.")
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))
    
    # display the tracker
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    x, y, y2, k = zip(*tracker)
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='best value'), secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', line={'width': 0.5}, name='iteration value'), secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=k, mode='lines', name='neighbourhood size'), secondary_y=True)
    fig.update_layout(
        title='Best solution found', template="presentation", xaxis_title="Iteration", yaxis_title="Distance")
    fig.show()
    
    return best_solution, min_distance_travelled

In [13]:
def OptimizationTSPTest(rg, iterations=5000):
    cities = simulateCities(rg, n_cities)
    inital_solution = list(range(len(cities))) + [0]
    print(cities)
    print("initial solution:", inital_solution)
    print("inital distance travelled:", evaluate(inital_solution, cities))
    drawSalesman(inital_solution, cities, "Initial Solution") # show the inital solution
    
    # Full enumeration section 
    print("Full Enumeration\n-----------")
    fe_solution, fe_distance = FullEnumeration(rg, cities, iterations=iterations)
    print("best solution:", solution)
    print("distance travelled:", fe_distance)
    drawSalesman(fe_solution, cities, "Full enumeration")
    
    # Greedy search algorithm section
    # Implement your greedy algorithm here
    print("Greedy Search:\n-----------")
    gs_solution, gs_distance = GreedySearch(rg, cities)
    drawSalesman(gs_solution, cities, "Greedy")
    print("greedy solution:", gs_solution)
    print("distance travelled:", gs_distance)
    
    print('\n\n\n')
    # drawSalesman(gs_solution, cities, "Greedy")
    
    # Local search
    # Implement your local search algorithm here
    print('Local Search\n------------')
    ls_solution, ls_distance = LocalSearch(rg, cities, iterations)
    drawSalesman(ls_solution, cities, "Local Search")
    print("best solution:", ls_solution)
    print("distance travelled:", ls_distance)
    
    print('\n\n\n')
    
    # Simulated annealing
    # Implement your simulated annealing here
    print('Simulated Annealing\n----------------')
    sa_solution, sa_distance = SimulatedAnnealing(rg, cities, iterations)
    drawSalesman(sa_solution, cities, "Simulated Annealing")
    print("best solution:", sa_solution)
    print("distance travelled:", sa_distance)
    
    print('\n\n\n')
    
    # VNS
    # Implement your VNS here
    print('Variable Neighborhood Search\n----------------')
    vns_solution, vns_distance = VNS(rg, cities, iterations=6000, K=50)
    drawSalesman(vns_solution, cities, "Variable Neighborhood Search")
    print("best solution:", vns_solution)
    print("distance travelled:", vns_distance)
    
    print('\n\n\n')

# run all
rg = Generator(PCG64(our_seed)) # use your own seed number here
OptimizationTSPTest(rg, iterations=5000)

{0: (0.52, 0.6), 1: (0.47, 0.2), 2: (0.53, 0.19), 3: (0.28, 0.75), 4: (0.55, 0.86), 5: (0.81, 0.25), 6: (0.19, 0.98), 7: (0.67, 0.28), 8: (0.2, 0.63), 9: (0.65, 0.9)}
initial solution: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
inital distance travelled: 5.276123160853705


Full Enumeration
-----------
running algorithm...
algorithm end.
time taken :0.22579s


best solution: [0, 7, 5, 2, 1, 8, 4, 9, 6, 3, 0]
distance travelled: 2.8749313329677086


Greedy Search:
-----------
running algorithm...
time taken :0.00030s


greedy solution: [0, 4, 9, 3, 8, 6, 1, 2, 7, 5, 0]
distance travelled: 2.9167482242863176




Local Search
------------
running algorithm...
algorithm end.
time taken :1.05178s


best solution: [0, 9, 4, 6, 3, 8, 1, 2, 5, 7, 0]
distance travelled: 2.556850390705624




Simulated Annealing
----------------
running algorithm...
algorithm end.
time taken :0.50116s


best solution: [0, 5, 7, 2, 1, 8, 3, 6, 4, 9, 0]
distance travelled: 2.538047925364124




Variable Neighborhood Search
----------------
running algorithm...
k reached
algorithm end.
time taken :0.02109s


best solution: [0, 5, 7, 2, 1, 9, 4, 6, 3, 8, 0]
distance travelled: 2.7475277934707134






# Lab 7 Optimization exercises: Multi-objective optimization

Implement the multi-objective optimization on the Knapsack problem using your choice of algorithm:
- simulated annealing
- variable neighbourhood search
- local search

## Knapsack problem

Each item has a weight $w$, utility $u$ and value $c$

objective functions:

$x_i=1$ if $i$ in knapsack, else $0$.

1. $\max \mathcal{f}(x)=\sum_{i=1}^n u_i x_i$ 
2. $\min \mathcal{f}(x)=\sum_{i=1}^n c_i x_i$
3. $w^Tx \leq W$ where $W$ is the maximum weight allowed: **150kg**

What is the set of items that can be put in the knapsack that is below the maximum weight and maximizes value and utility?

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd

from numpy.random import Generator, PCG64

In [ ]:
# objective functions
def total_value(solution):
    return np.sum(item_value[solution])

def total_utility(solution):
    return np.sum(item_utility[solution])

# constraints on weight, returns True if items below max_weight, else false
def weight_limit(solution, max_weight):
    total_weight = np.sum(item_weights[solution])
    return total_weight < max_weight

def weight(solution):
    return np.sum(item_weights[solution])

In [ ]:
# Pareto dominance
def improvement(cost_old, cost_new):
    is_dominating = 0 
    rule_1 = []
    rule_2 = []
    
    for i, _ in enumerate(cost_old):
        if cost_old[i] <= cost_new[i]:
            rule_1.append(True)
        else:
            rule_1.append(False)
        if cost_old[i] < cost_new[i]:
            rule_2.append(True)
        else:
            rule_2.append(False)
    
    if all(rule_1) and any(rule_2):
        is_dominating = 1
    
    return is_dominating

def dominance(P, cost_new):
    D = []
    S = []
    for i, p in enumerate(P):
        cost = p['cost']
        if improvement(cost, cost_new):
            D.append(i)
        if improvement(cost_new, cost):
            S.append(i)
    return D, S

In [ ]:
# Example: full enumeration algorithm

def knapsack_fe(rg, solution):
    n = rg.choice(range(1, 20)) # random number of items (up to 20 items)
    new_solution = rg.choice(range(20), n, replace=False) # pick n items from [1,...,20]

    while not weight_limit(new_solution, W): # pick another solution if new_solution is above limit
        n = rg.choice(range(1, 20))
        new_solution = rg.choice(range(20), n, replace=False)
    
    return new_solution

In [ ]:
def main(rg, iterations=1000, W=150):
    # simulate some random values
    item_weights = rg.integers(10,50,20)
    item_utility = rg.integers(0,100,20)
    item_value = rg.random(20)

    P = [] # initialize empty lists to store solutions and costs
    Q = []
    R = []
    
    # generate an initial random solution and add it to P
    # example start with 2 random item from the set
    solution = list(rg.choice(len(item_weights), 2, replace=False)) 
    cost = (total_value(solution), total_utility(solution))
    
    # add to P as a dictionary of values
    P.append({'solution': solution, 'cost': cost, 'weight': weight(solution)}) 
    
    # Main iteration
    for i in range(1, iterations):
        
        # select a random solution set from set P
        p = rg.choice(P)
        
        #################################
        # Implement your algorithm here #
        #################################
        
        # new_solution = knapsack_fe(rg, solution)
        
        #################################
        
        # compute the new cost of the new solution
        new_cost = (total_value(new_solution), total_utility(new_solution))
        
        # Pareto dominance algorithm
        D, S = dominance(P, new_cost)
        
        # Pareto algorithm update
        if len(S) == 0:
            # add items p in P to list R for all items in D and not already in R
            R.extend([p for n, p in enumerate(P) if (n in D and p not in R)])
            # remove items in P
            P = [p for n, p in enumerate(P) if n not in D]
            # finally add new_solution to P
            P.append({'solution': list(new_solution), 'cost': new_cost, 'weight': weight(new_solution)})
        else:
            # track considered solutions
            Q.append({'solution': list(new_solution), 'cost': new_cost, 'weight': weight(new_solution)})
            if len(Q) > 100:
                Q.pop(0) # we store only the last 100 considered values for memory savings
    
    print('optimization complete')
    # return the pareto set, considered set and removed set
    return P, Q, R

# run
rg = Generator(PCG64(42069)) # set your own unique seed number
P, Q, R = main(rg, iterations=1000, W=150)

## Display the output

In [ ]:
fig = go.Figure()

c = pd.DataFrame(P)
x, y = list(zip(*c['cost']))
fig.add_trace(go.Scatter(x=x, y=y, mode='markers+text', name='pareto', 
    hovertemplate ='items: [%{hovertext}]', hovertext=c['solution'],
    text=c['weight'], texttemplate = "%{text} kg", textposition="top center", textfont = {'size': 12}))

c = pd.DataFrame(R)
x, y = list(zip(*c['cost']))
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='removed',
    hovertemplate ='items: [%{hovertext}]', hovertext=c['solution'],))

c = pd.DataFrame(Q)
x, y = list(zip(*c['cost']))
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='considered',
    hovertemplate ='items: [%{hovertext}]', hovertext=c['solution'],))

fig.update_layout(template="presentation", xaxis_title="Value", yaxis_title="Utility", width=800, height=600)
fig.show()